In [1]:
import tushare as ts
import sina_data
import pandas as pd
from pandas import DataFrame, Series
from datetime import datetime, timedelta
from dateutil.parser import parse
import time
import common_util
import os

In [2]:
# 日K数据判断是否开板
def is_sold(code, start_date):
    print(code)
    try:
        time.sleep(1)
        pro = ts.pro_api('ba73b3943bdd57c2ff05991f7556ef417f457ac453355972ff5d01ce')
        start_date = (parse(str(start_date)) + timedelta(1)).strftime('%Y%m%d')
        end_date = datetime.now().strftime('%Y%m%d')
        daily_k = pro.daily(ts_code=code, start_date=start_date, end_date=end_date)
        if len(daily_k) > 0:
            daily_k['flag'] = daily_k.apply(
                lambda x: x['high'] == x['low'] and x['open'] == x['close'] and x['high'] == x['low'],
                axis=1
            )
            flag = daily_k['flag'].sum()
            result = True
            for each in daily_k['flag'].tolist():
                result = result and each
            return result
        else:
            return True
    except Exception as e:
        print('再次请求ts数据')
        time.sleep(1)
        a = is_sold(code, start_date)
        return a

a = is_sold('300843.sz', '2020-10-13')
print(a)

300843.sz
False


In [5]:
def get_float_share(code):
    print(code)
    try:
        time.sleep(1)
        pro = ts.pro_api('ba73b3943bdd57c2ff05991f7556ef417f457ac453355972ff5d01ce')
        # target_date = datetime.now().strftime('%Y%m%d')
        target_data = []
        delta = 0
        count = 1
        while len(target_data) == 0:
            target_date = datetime.now()+timedelta(delta)
            target_data = pro.daily_basic(
                ts_code=code, trade_date=target_date.strftime('%Y%m%d'), fields='free_share'
            )
            delta = delta - 1
            time.sleep(0.5)
            count = count + 1
            if count > 3:
                return 1000000
        return target_data.values[0][0] * 10000

    except Exception as e:
        time.sleep(1)
        get_float_share(code)
        print('再次请求ts数据.....')

In [7]:
pro = ts.pro_api('ba73b3943bdd57c2ff05991f7556ef417f457ac453355972ff5d01ce')
basic_data = pro.stock_basic()
print('股票筛选')
# basic_data.to_excel(r'C:\Users\duanp\Desktop\test\stock_basic.xlsx')
# basic_data = pd.read_excel(r'C:\Users\duanp\Desktop\test\stock_basic.xlsx')
# 筛选上市日期为近一月的股票
start_date = datetime.now() + timedelta(-30)
end_date = datetime.now() + timedelta(1)
basic_data['list_date'] = basic_data['list_date'].apply(lambda x: parse(str(x)))
basic_data = basic_data[basic_data['list_date'] > start_date]
basic_data = basic_data[basic_data['list_date'] < end_date]
# 剔除科创板股票
basic_data = basic_data[basic_data['market'] != '科创板']

股票筛选


In [8]:
basic_data

,ts_code,symbol,name,area,industry,market,list_date
1417,003001.SZ,003001,中岩大地,北京,建筑工程,中小板,2020-10-13
1427,003012.SZ,003012,东鹏控股,广东,陶瓷,中小板,2020-10-19
1428,003013.SZ,003013,地铁设计,广东,建筑工程,中小板,2020-10-22
1429,003015.SZ,003015,日久光电,江苏,元器件,中小板,2020-10-21
1430,003016.SZ,003016,欣贺股份,福建,服饰,中小板,2020-10-26
1431,003017.SZ,003017,大洋生物,浙江,化工原料,中小板,2020-10-26
2296,300898.SZ,300898,熊猫乳品,浙江,乳制品,创业板,2020-10-16
2297,300899.SZ,300899,上海凯鑫,上海,环境保护,创业板,2020-10-16
2298,300900.SZ,300900,C广联,黑龙江,航空,创业板,2020-10-29
2299,300901.SZ,300901,C中胤,浙江,服饰,创业板,2020-10-29


In [9]:
basic_data['target_flag'] = basic_data.apply(lambda x: is_sold(x['ts_code'], x['list_date']), axis=1)
basic_data = basic_data[basic_data['target_flag']]

003001.SZ
003012.SZ
003013.SZ
003015.SZ
003016.SZ
003017.SZ
300898.SZ
300899.SZ
300900.SZ
300901.SZ
300902.SZ
300999.SZ
601187.SH
601568.SH
601995.SH
605058.SH
605169.SH
605336.SH
605338.SH


In [10]:
basic_data

,ts_code,symbol,name,area,industry,market,list_date,target_flag


In [6]:
pro = ts.pro_api('ba73b3943bdd57c2ff05991f7556ef417f457ac453355972ff5d01ce')
basic_data = pro.stock_basic()
print('股票筛选')
# basic_data.to_excel(r'C:\Users\duanp\Desktop\test\stock_basic.xlsx')
# basic_data = pd.read_excel(r'C:\Users\duanp\Desktop\test\stock_basic.xlsx')
# 筛选上市日期为近一月的股票
start_date = datetime.now() + timedelta(-30)
end_date = datetime.now() + timedelta(1)
basic_data['list_date'] = basic_data['list_date'].apply(lambda x: parse(str(x)))
basic_data = basic_data[basic_data['list_date'] > start_date]
basic_data = basic_data[basic_data['list_date'] < end_date]
# 剔除科创板股票
basic_data = basic_data[basic_data['market'] != '科创板']
# 筛选未开板的股票
basic_data['target_flag'] = basic_data.apply(lambda x: is_sold(x['ts_code'], x['list_date']), axis=1)
basic_data = basic_data[basic_data['target_flag']]
print('补充流通股本数据')
# 补充流通股本信息
basic_data['float_share'] = basic_data.apply(lambda x: get_float_share(x['ts_code']), axis=1)
basic_data['float_share'] = basic_data['float_share'].fillna('100000')
print('预警股票如下：')
print(basic_data)

股票筛选
003001.SZ
003012.SZ
003013.SZ
003015.SZ
003016.SZ
003017.SZ
300898.SZ
300899.SZ
300900.SZ
300901.SZ
300902.SZ
300999.SZ
601187.SH
601568.SH
601995.SH
605058.SH
605169.SH
605336.SH
605338.SH
补充流通股本数据


ValueError: Wrong number of items passed 8, placement implies 1

In [ ]:
basic_data['target_code'] = basic_data['ts_code'].apply(lambda x: common_util.get_format_code(x, 'num'))


In [ ]:
tick_data = sina_data.get_tick_data(basic_data['symbol'].to_list())

In [ ]:
tick_data['股票代码'] = tick_data['股票代码'].apply(lambda x: common_util.get_format_code(x, 'wind'))

In [ ]:
tick_data = tick_data[['股票代码', '买一量']]

In [ ]:
temp_data = basic_data.merge(tick_data, left_on='ts_code', right_on='股票代码')

In [ ]:
temp_data

In [ ]:
temp_data['flag'] = temp_data.apply(lambda x: float(x['买一量']) < float(x['float_share']) * 0.03, axis=1)

In [ ]:
temp_data

In [ ]:
temp_data = temp_data[temp_data['flag']]

In [ ]:
temp_data

In [ ]:
print(temp_data)